In [2]:
import pandas as pd
import numpy as np
import anndata as ad

#### TCGA-BRCA (The Cancer Genome Atlas - Breast Cancer)

In [ ]:
tcga = pd.read_csv(bkdata_path, sep='\t')
tcga = tcga.sort_index()
tcga.to_csv('../DATA/TCGA/TCGA_GDC_HTSeq_TPM.csv')

In [ ]:
abis = pd.read_csv('../DATA/GSE107011/GSE107011_Processed_data_TPM.txt')
epic = pd.read_csv('../DATA/EPIC/melanoma_counts.csv')

#### METABRIC (Molecular 

In [ ]:
metabric = pd.read_csv('../DATA/METABRIC/data_expression_median.txt', sep='\t', index_col=0)
metabric.index.name = None
metabric = metabric.iloc[:,1:]
metabric = metabric.sort_index()
metabric.to_csv('../DATA/METABRIC/METABRIC.csv')

In [57]:
sdy67_1 = pd.read_csv('../DATA/SDY67/SDY67_EXP13377_RNA_seq.703318.tsv', sep='\t', index_col=0)
sdy67_2 = pd.read_csv('../DATA/SDY67/SDY67_EXP14625_RNA_seq.703317.tsv', sep='\t', index_col=0)
sdy67_1.index.name = None
sdy67_2.index.name = None
sdy67 = sdy67_1.join(sdy67_2)
sdy67 = sdy67.sort_index()
sdy67_meta = pd.read_csv('../DATA/SDY67/SDY67-DR34_Subject_2_RNA_sequencing_result.txt', sep='\t')
sdy67_meta['SubjectID'] = [sub+'_'+str(time) for sub, time in zip(list(sdy67_meta['Subject Accession'].values),list(sdy67_meta['Study Time Collected'].values))]
sdy67_id = sdy67_meta.reset_index().set_index('Expsample Accession').loc[sdy67.columns.values.tolist(),'SubjectID'].values.tolist()
sdy67.columns = sdy67_id

In [60]:
sdy67

,SUB113487_3,SUB113588_0,SUB113476_0,SUB113473_3,SUB113568_3,SUB113498_3,SUB113571_0,SUB113458_3,SUB113588_28,SUB113548_0,...,SUB113482_3,SUB113467_0,SUB113590_3,SUB113586_0,SUB113584_28,SUB113512_3,SUB113497_28,SUB113528_0,SUB113598_0,SUB113542_3
1/2-SBSRNA4,25,40,25,16,35,37,20,33,10,38,...,47,60,52,62,58,56,53,62,65,57
A1BG,451,456,232,251,261,284,497,240,358,278,...,699,1093,1140,1560,792,1059,834,1077,941,1316
A1BG-AS1,81,60,59,63,74,76,69,63,60,86,...,114,204,147,157,164,169,155,126,98,180
A1CF,1,1,0,1,3,2,1,2,1,3,...,2,13,1,4,2,2,4,1,1,3
A2LD1,377,354,258,289,265,173,277,177,393,207,...,457,712,878,572,606,559,676,731,540,751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYX,26425,39501,29838,24642,20993,17827,27698,19663,32972,19790,...,45453,62063,77391,65860,61704,67435,50247,79615,69563,78636
ZZEF1,8423,7531,14192,10497,14426,14006,6538,12165,8685,10151,...,26621,29445,21847,16682,24035,16315,24411,17803,22700,16971
ZZZ3,3255,3152,3688,3599,3601,3388,2358,3684,3368,3937,...,6798,7399,6046,4928,6630,4901,7848,4915,5142,4257
psiTPTE22,56,286,241,238,75,99,230,123,171,106,...,405,338,468,1430,311,393,574,506,228,520


In [69]:
# sdy67_label = pd.read_csv('../DATA/SDY67/SDY67_250_label.csv', index_col=0) #'../DATA/SDY67/SDY67_extracted_from_mmc7.csv', index_col=0)
sdy67_label = pd.read_csv('../DATA/SDY67/SDY67_raw_labels_from_mmc7.csv', index_col=0) #'../DATA/SDY67/SDY67_extracted_from_mmc7.csv', index_col=0)
sdy67_label.index.name = None
sdy67_label['Other'] = 0
# sdy67_label = sdy67_label.fillna(0)
sdy67_label = sdy67_label.dropna()
sdy67_label = sdy67_label.reindex(sdy67.columns)
for i in range(len(sdy67_label)):
    sumval = np.sum(sdy67_label.iloc[i,:])
    if sumval >= 100.:
        sdy67_label.iloc[i,:] = sdy67_label.iloc[i,:]/sumval
#     elif sumval == 0.:
#         sdy67_label.iloc[i,:] = None
    else:
        sdy67_label.iloc[i,-1] = 100.-sumval
        sdy67_label.iloc[i,:] = sdy67_label.iloc[i,:]/100.

sdy67_label = sdy67_label.dropna()

In [70]:
sdy67_label

,B Naive,B Ex,B NSM,B SM,Plasmablasts,T CD4,T CD8,Basophils LD,mDCs,pDCs,Monocytes C,Monocytes I,Monocytes NC,NK,Other
SUB113588_0,0.014000,0.002800,0.011900,0.007800,0.000180,0.468000,0.206000,0.000730,0.008700,0.00180,0.138000,0.015700,0.020000,0.049600,0.054790
SUB113476_0,0.017101,0.002736,0.026579,0.005179,0.000254,0.361558,0.213026,0.000674,0.014853,0.00469,0.238433,0.025993,0.035276,0.053647,0.000000
SUB113473_3,0.006200,0.003100,0.008300,0.013000,0.000380,0.388000,0.098300,0.000360,0.026900,0.00190,0.141000,0.027500,0.020300,0.134000,0.130760
SUB113498_3,0.015700,0.003000,0.007600,0.006900,0.000170,0.365000,0.121000,0.000650,0.024100,0.00350,0.226000,0.016200,0.012400,0.094400,0.103380
SUB113571_0,0.008000,0.002900,0.016400,0.014700,0.000230,0.364000,0.182000,0.000350,0.051900,0.00620,0.195000,0.014700,0.020300,0.081200,0.042120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SUB113590_3,0.026000,0.013400,0.003500,0.010900,0.000033,0.329000,0.119000,0.000370,0.011900,0.00390,0.113000,0.012300,0.012700,0.187000,0.156997
SUB113586_0,0.009100,0.013600,0.000170,0.001300,0.000011,0.207000,0.082300,0.000092,0.008700,0.00120,0.077400,0.026700,0.007500,0.129000,0.435927
SUB113497_28,0.007600,0.004000,0.000880,0.002100,0.000029,0.328000,0.065000,0.000420,0.036400,0.00380,0.139000,0.012400,0.010200,0.309000,0.081171
SUB113528_0,0.004600,0.007400,0.010700,0.015000,0.000330,0.301000,0.235000,0.000350,0.003900,0.00180,0.126000,0.022200,0.023300,0.231000,0.017420


In [61]:
sdy67 = sdy67.loc[:,sdy67_label.index]

In [62]:
sdy67

,SUB113487_3,SUB113588_0,SUB113476_0,SUB113473_3,SUB113568_3,SUB113498_3,SUB113571_0,SUB113458_3,SUB113588_28,SUB113548_0,...,SUB113482_3,SUB113467_0,SUB113590_3,SUB113586_0,SUB113584_28,SUB113512_3,SUB113497_28,SUB113528_0,SUB113598_0,SUB113542_3
1/2-SBSRNA4,25,40,25,16,35,37,20,33,10,38,...,47,60,52,62,58,56,53,62,65,57
A1BG,451,456,232,251,261,284,497,240,358,278,...,699,1093,1140,1560,792,1059,834,1077,941,1316
A1BG-AS1,81,60,59,63,74,76,69,63,60,86,...,114,204,147,157,164,169,155,126,98,180
A1CF,1,1,0,1,3,2,1,2,1,3,...,2,13,1,4,2,2,4,1,1,3
A2LD1,377,354,258,289,265,173,277,177,393,207,...,457,712,878,572,606,559,676,731,540,751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYX,26425,39501,29838,24642,20993,17827,27698,19663,32972,19790,...,45453,62063,77391,65860,61704,67435,50247,79615,69563,78636
ZZEF1,8423,7531,14192,10497,14426,14006,6538,12165,8685,10151,...,26621,29445,21847,16682,24035,16315,24411,17803,22700,16971
ZZZ3,3255,3152,3688,3599,3601,3388,2358,3684,3368,3937,...,6798,7399,6046,4928,6630,4901,7848,4915,5142,4257
psiTPTE22,56,286,241,238,75,99,230,123,171,106,...,405,338,468,1430,311,393,574,506,228,520


In [13]:
sdy67.to_csv('../DATA/SDY67/SDY67_468_pp.csv')
sdy67_label.to_csv('../DATA/SDY67/SDY67_468_label_allct_pp.csv')

In [71]:
sdy67_label2 =  pd.DataFrame()
sdy67_label2['B cell'] = sdy67_label['B Naive'] + sdy67_label['B Ex'] + sdy67_label['B NSM'] + sdy67_label['B SM'] + sdy67_label['Plasmablasts']
sdy67_label2['T CD4'] = sdy67_label['T CD4']
sdy67_label2['T CD8'] = sdy67_label['T CD8']
sdy67_label2['NK'] = sdy67_label['NK']
sdy67_label2['Myeloid'] = sdy67_label['Basophils LD'] + sdy67_label['mDCs'] + sdy67_label['pDCs'] + sdy67_label['Monocytes C'] + sdy67_label['Monocytes I'] + sdy67_label['Monocytes NC']
sdy67_label2['Other'] = sdy67_label['Other']


In [72]:
sdy67_label2

,B cell,T CD4,T CD8,NK,Myeloid,Other
SUB113588_0,0.036680,0.468000,0.206000,0.049600,0.184930,0.054790
SUB113476_0,0.051849,0.361558,0.213026,0.053647,0.319920,0.000000
SUB113473_3,0.030980,0.388000,0.098300,0.134000,0.217960,0.130760
SUB113498_3,0.033370,0.365000,0.121000,0.094400,0.282850,0.103380
SUB113571_0,0.042230,0.364000,0.182000,0.081200,0.288450,0.042120
...,...,...,...,...,...,...
SUB113590_3,0.053833,0.329000,0.119000,0.187000,0.154170,0.156997
SUB113586_0,0.024181,0.207000,0.082300,0.129000,0.121592,0.435927
SUB113497_28,0.014609,0.328000,0.065000,0.309000,0.202220,0.081171
SUB113528_0,0.038030,0.301000,0.235000,0.231000,0.177550,0.017420


In [73]:
# sdy67.to_csv('../DATA/SDY67/SDY67_468_pp.csv')
sdy67_label2.to_csv('../DATA/SDY67/SDY67_250_label_6ct_pp.csv')

-----------------------------------------------------------------------------

In [32]:
y = ad.read_h5ad('C:/Users/yw_ji/Documents/MSc Thesis/DATA/scaden_input/simulated_data/data_10000.h5ad')
y_data = y_data.reindex(y.var.index)

In [34]:
sdy67_ad = ad.AnnData(sdy67.T, obs=sdy67_label)
sdy67_ad.uns['cell_types'] = sdy67_label.columns.tolist()
sdy67_ad = sdy67_ad[:,y.var.index]

In [35]:
sdy67_ad.write_h5ad('C:/Users/yw_ji/Documents/MSc Thesis/DATA/scaden_input/SDY67_250.h5ad')

In [38]:
sdy67_ad.var_names

Index(['AARSD1', 'ABCA10', 'ACBD3', 'ACSL1', 'ACSM3', 'ACSS2', 'ADAM12',
       'ADAM15', 'AEBP1', 'AFAP1',
       ...
       'VMP1', 'WARS', 'WIPI1', 'YES1', 'ZBTB16', 'ZFAND2A', 'ZNF165',
       'ZNF296', 'ZNF683', 'ZNF80'],
      dtype='object', length=526)

In [10]:
CELLTYPES = ['CAF','B_cell','Cancer','Endothelial','Myeloid','Other_T','CD8+_T_cell',
             'NK_cell','CD4+_T_cell','Treg','PVL','Epithelial']
sdy67_label.dropna().index

Index(['SUB113487_3', 'SUB113588_0', 'SUB113476_0', 'SUB113473_3',
       'SUB113568_3', 'SUB113498_3', 'SUB113571_0', 'SUB113458_3',
       'SUB113588_28', 'SUB113548_0',
       ...
       'SUB113482_3', 'SUB113467_0', 'SUB113590_3', 'SUB113586_0',
       'SUB113584_28', 'SUB113512_3', 'SUB113497_28', 'SUB113528_0',
       'SUB113598_0', 'SUB113542_3'],
      dtype='object', length=468)

In [15]:
# ABIS dataset need to load genes.tsv then match Ensembl id to gene id 
# EPIC dataset needs fixing
# abis = pd.read_csv('../DATA/GSE107011/GSE107011_Processed_data_TPM.txt', index_col=0) # EnsDb.Hsapiens.v79, aggregated signals from duplicate probes by max value
abis = pd.read_csv('../DATA/GSE107011/GSE107011_Processed_data_TPM_HGNC_Gene_ID.csv', index_col=0)
abis_label = pd.read_csv('../DATA/GSE107011/abis_raw_labels_from_mmc7.csv', index_col=0)
# abis_label = pd.read_csv('../DATA/GSE107011/GSE107011_label_7ct.csv', index_col=0)

In [16]:
for i in range(len(abis_label)):
    sumval = np.sum(abis_label.iloc[i,:])
    if sumval >= 100.:
        abis_label.iloc[i,:] = abis_label.iloc[i,:]/sumval
    else:
        abis_label.iloc[i,-1] += 100.-sumval
        abis_label.iloc[i,:] = abis_label.iloc[i,:]/100.

In [17]:
abis = abis.drop(columns='DZQV_PBMC')
abis_pbmc = abis.filter(regex='_PBMC$', axis=1)
abis_pbmc_label = abis_label

In [68]:
### June 13, 2021, ABIS 6 celltypes
abis_pbmc_label2 =  pd.DataFrame()
abis_pbmc_label2['B cell'] = abis_pbmc_label['B Naive'] + abis_pbmc_label['B Ex'] + abis_pbmc_label['B NSM'] + abis_pbmc_label['B SM'] + abis_pbmc_label['Plasmablasts']
abis_pbmc_label2['T CD4'] = abis_pbmc_label['T CD4']
abis_pbmc_label2['T CD8'] = abis_pbmc_label['T CD8']
abis_pbmc_label2['NK'] = abis_pbmc_label['NK']
abis_pbmc_label2['Myeloid'] = abis_pbmc_label['Basophils LD'] + abis_pbmc_label['mDCs'] + abis_pbmc_label['pDCs'] + abis_pbmc_label['Monocytes C'] + abis_pbmc_label['Monocytes I'] + abis_pbmc_label['Monocytes NC']
abis_pbmc_label2['Other'] = abis_pbmc_label['Other']
abis_pbmc_label2.to_csv('../DATA/GSE107011/abis_pbmc_label_6ct.csv')

In [20]:
sdy67_label.columns

Index(['B Naive', 'B Ex', 'B NSM', 'B SM', 'Plasmablasts', 'T CD4', 'T CD8',
       'Basophils LD', 'mDCs', 'pDCs', 'Monocytes C', 'Monocytes I',
       'Monocytes NC', 'NK', 'Other'],
      dtype='object')

In [21]:
abis_label.columns

Index(['Progenitors', 'B Naive', 'B NSM', 'B SM', 'B Ex', 'Plasmablasts',
       'T CD4 Naive', 'Tregs', 'Tfh', 'Th1', 'Th1/Th17', 'Th17', 'Th2',
       'T CD4 TE', 'T CD8 Naive', 'T CD8 CM', 'T CD8 EM', 'T CD8 TE',
       'T gd Vd2', 'T gd non-Vd2', 'MAIT', 'NK', 'pDCs', 'mDCs', 'Monocytes C',
       'Monocytes I', 'Monocytes NC', 'Neutrophils LD', 'Basophils LD'],
      dtype='object')

In [26]:
pd.Series([x[5:] for x in abis.columns.tolist()]).unique()

array(['CD8_naive', 'CD8_CM', 'CD8_EM', 'CD8_TE', 'MAIT', 'VD2+', 'VD2-',
       'TFH', 'Treg', 'Th1', 'Th1/Th17', 'Th17', 'Th2', 'CD4_naive',
       'Progenitor', 'B_naive', 'B_NSM', 'B_Ex', 'B_SM', 'Plasmablasts',
       'C_mono', 'I_mono', 'NC_mono', 'NK', 'pDC', 'mDC', 'Neutrophils',
       'Basophils', 'CD4_TE', 'PBMC'], dtype=object)

In [30]:
abis_label

,Progenitors,B Naive,B NSM,B SM,B Ex,Plasmablasts,T CD4 Naive,Tregs,Tfh,Th1,...,T gd non-Vd2,MAIT,NK,pDCs,mDCs,Monocytes C,Monocytes I,Monocytes NC,Neutrophils LD,Basophils LD
Sample Name,,,,,,,,,,,,,,,,,,,,,
453W,0.00091,0.02750,0.01560,0.0072,0.00093,0.00066,0.09165,0.01741,0.02835,0.01845,...,0.00727,0.02047,0.0678,0.00570,0.0038,0.194,0.0141,0.0085,0.0122,0.06763
684C,0.00043,0.06740,0.01160,0.0106,0.00270,0.00740,0.10051,0.02222,0.05073,0.03370,...,0.00979,0.03449,0.0845,0.00610,0.0066,0.196,0.0087,0.0172,0.0187,0.06464
CR3L,0.00065,0.08380,0.01020,0.0111,0.00100,0.00540,0.10871,0.02093,0.02570,0.01768,...,0.00635,0.05075,0.1080,0.00550,0.0076,0.140,0.0083,0.0113,0.0598,0.04173
FLWA,0.00220,0.05450,0.01010,0.0158,0.00180,0.00150,0.06084,0.02019,0.03971,0.02598,...,0.00218,0.03692,0.1950,0.00420,0.0064,0.196,0.0047,0.0079,0.0362,0.07061
FY2H,0.00100,0.06020,0.00390,0.0043,0.00087,0.00110,0.08098,0.01773,0.03136,0.01491,...,0.10916,0.02188,0.0260,0.00370,0.0099,0.268,0.0455,0.0132,0.0220,0.03824
CYFZ,0.00068,0.05690,0.01080,0.0122,0.00200,0.00180,0.05531,0.01545,0.02423,0.01404,...,0.01715,0.01110,0.1400,0.00280,0.0038,0.111,0.0289,0.0280,0.0120,0.10013
925L,0.00046,0.12734,0.01151,0.0156,0.00127,0.00111,0.03276,0.01556,0.04366,0.02322,...,0.00963,0.01053,0.0969,0.00099,0.0056,0.170,0.0119,0.0147,0.0239,0.09335
9JD4,0.00065,0.04700,0.01120,0.0177,0.00440,0.00270,0.11561,0.02425,0.04762,0.03449,...,0.02206,0.03362,0.0753,0.00480,0.0048,0.146,0.0113,0.0125,0.0060,0.04568
G4YW,0.00110,0.03640,0.01310,0.0181,0.00280,0.00460,0.09348,0.02442,0.04086,0.01757,...,0.02183,0.04317,0.1750,0.00270,0.0055,0.186,0.0175,0.0167,0.0148,0.03759


In [39]:
row_names = []
values = []

for i, j in enumerate(abis.columns):
    if j[5:] == 'B_naive':
        celltype = 'B Naive'
        idx = 0
    elif j[5:] == 'B_Ex':
        celltype = 'B Ex'
        idx = 1   
    elif j[5:] == 'B_NSM':
        celltype = 'B NSM'
        idx = 2
    elif j[5:] == 'B_SM':
        celltype = 'B SM'
        idx = 3
    elif j[5:] == 'Plasmablasts':
        celltype = 'Plasmablasts'
        idx = 4
    elif j[5:] in ['TFH','Treg','Th1','Th1/Th17','Th17','Th2','CD4_naive', 'CD4_TE']:
        celltype = 'T CD4'
        idx = 5
    elif j[5:] in ['CD8_naive','CD8_CM','CD8_EM','CD8_TE']:
        celltype = 'T CD8'
        idx = 6
    elif j[5:] == 'Basophils':
        celltype = 'Basophils LD'
        idx = 7
    elif j[5:] == 'mDC':
        celltype = 'mDCs'
        idx = 8
    elif j[5:] == 'pDC':
        celltype = 'pDCs'
        idx = 9
    elif j[5:] == 'C_mono':
        celltype = 'Monocytes C'
        idx = 10
    elif j[5:] == 'I_mono':
        celltype = 'Monocytes I'
        idx = 11
    elif j[5:] == 'NC_mono':
        celltype = 'Monocytes NC'
        idx = 12
    elif j[5:] == 'NK':
        celltype = 'NK'
        idx = 13
    elif j[5:] in ['MAIT','VD2+','VD2-','Progenitor','Neutrophils']:
        celltype = 'Other'
        idx = 14
    else:
        celltype = 'PBMC'
        value = [abis_label.loc[[j[:4]],['B Naive']].values.sum(),
                 abis_label.loc[[j[:4]],['B Ex']].values.sum(),
                 abis_label.loc[[j[:4]],['B NSM']].values.sum(),
                 abis_label.loc[[j[:4]],['B SM']].values.sum(),
                 abis_label.loc[[j[:4]],['Plasmablasts']].values.sum(),
                 abis_label.loc[[j[:4]],['Tfh','Tregs','Th1','Th1/Th17','Th17','Th2','T CD4 Naive', 'T CD4 TE']].values.sum(),
                 abis_label.loc[[j[:4]],['T CD8 Naive','T CD8 CM','T CD8 EM','T CD8 TE']].values.sum(),
                 abis_label.loc[[j[:4]],['Basophils LD']].values.sum(),
                 abis_label.loc[[j[:4]],['mDCs']].values.sum(),
                 abis_label.loc[[j[:4]],['pDCs']].values.sum(),
                 abis_label.loc[[j[:4]],['Monocytes C']].values.sum(),
                 abis_label.loc[[j[:4]],['Monocytes I']].values.sum(),
                 abis_label.loc[[j[:4]],['Monocytes NC']].values.sum(),
                 abis_label.loc[[j[:4]],['NK']].values.sum(),
                 abis_label.loc[[j[:4]],['MAIT','T gd Vd2','T gd non-Vd2','Progenitors','Neutrophils LD']].values.sum(),
                ]

    if j == 'DZQV_PBMC':
        pass
    else:
        sample_name = j
        row_names.append(sample_name)
        tmp = [0]*15
        if celltype == 'PBMC':
            values.append(value)
        else:
            tmp[idx] = 1
            values.append(tmp)

abis_label2 = pd.DataFrame(data=values, index=row_names, columns=sdy67_label.columns)

In [111]:
row_names = []
values = []

for i, j in enumerate(abis.columns):
    if j[5:] in ['B_naive','B_NSM','B_Ex','B_SM','Plasmablasts']:
        celltype = 'B_cell'
        idx = 0
    elif j[5:] in ['TFH','Treg','Th1','Th1/Th17','Th17','Th2','CD4_naive', 'CD4_TE']:
        celltype = 'T_CD4'
        idx = 1
    elif j[5:] in ['CD8_naive','CD8_CM','CD8_EM','CD8_TE']:
        celltype = 'T_CD8'
        idx = 2
    elif j[5:] in ['pDC', 'mDC']:
        celltype = 'DCs'
        idx = 3
    elif j[5:] in ['C_mono','I_mono','NC_mono']:
        celltype = 'Monocytes'
        idx = 4
    elif j[5:] in ['NK']:
        celltype = 'NK'
        idx = 5
    elif j[5:] in ['MAIT','VD2+','VD2-','Progenitor','Neutrophils','Basophils']:
        celltype = 'Other'
        idx = 6
    else:
        celltype = 'PBMC'

    if j == 'DZQV_PBMC':
        pass
    else:
        sample_name = j
        row_names.append(sample_name)
        tmp = [0]*7
        if celltype == 'PBMC':
            values.append(abis_label.loc[[sample_name]].values.squeeze().tolist())
        else:
            tmp[idx] = 1
            values.append(tmp)

abis_label = pd.DataFrame(data=values, index=row_names, columns=abis_label.columns)

In [40]:
abis_label

,B Naive,B Ex,B NSM,B SM,Plasmablasts,T CD4,T CD8,Basophils LD,mDCs,pDCs,Monocytes C,Monocytes I,Monocytes NC,NK,Other
DZQV_CD8_naive,0.0000,0.00000,0.0000,0.0000,0.0000,0.00000,1.00000,0.00000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.00000
DZQV_CD8_CM,0.0000,0.00000,0.0000,0.0000,0.0000,0.00000,1.00000,0.00000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.00000
DZQV_CD8_EM,0.0000,0.00000,0.0000,0.0000,0.0000,0.00000,1.00000,0.00000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.00000
DZQV_CD8_TE,0.0000,0.00000,0.0000,0.0000,0.0000,0.00000,1.00000,0.00000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.00000
DZQV_MAIT,0.0000,0.00000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9JD4_PBMC,0.0470,0.00440,0.0112,0.0177,0.0027,0.30033,0.23596,0.04568,0.0048,0.0048,0.146,0.0113,0.0125,0.0753,0.08033
G4YW_PBMC,0.0364,0.00280,0.0131,0.0181,0.0046,0.24426,0.13448,0.03759,0.0055,0.0027,0.186,0.0175,0.0167,0.1750,0.10527
4DUY_PBMC,0.0519,0.00098,0.0120,0.0088,0.0061,0.24222,0.12392,0.10206,0.0056,0.0014,0.173,0.0059,0.0436,0.0792,0.14332
36TS_PBMC,0.0415,0.00210,0.0107,0.0131,0.0018,0.19150,0.17041,0.08545,0.0057,0.0051,0.247,0.0333,0.0350,0.0772,0.08014


In [41]:
# abis.to_csv('../DATA/GSE107011/abis_data.csv')
abis_label.to_csv('../DATA/GSE107011/abis_label_15ct.csv')

In [52]:
abis_pbmc_label['T CD4'] = abis_pbmc_label.loc[:,['Tfh','Tregs','Th1','Th1/Th17','Th17','Th2','T CD4 Naive', 'T CD4 TE']].sum(axis=1)
abis_pbmc_label['T CD8'] = abis_pbmc_label.loc[:,['T CD8 Naive','T CD8 CM','T CD8 EM','T CD8 TE']].sum(axis=1)
abis_pbmc_label['Other'] = abis_pbmc_label.loc[:,['MAIT','T gd Vd2','T gd non-Vd2','Progenitors','Neutrophils LD']].sum(axis=1)
abis_pbmc_label = abis_pbmc_label.loc[:,['B Naive','B Ex','B NSM','B SM','Plasmablasts','T CD4','T CD8','Basophils LD','mDCs','pDCs','Monocytes C','Monocytes I','Monocytes NC','NK','Other']]

In [53]:
# abis_pbmc.to_csv('../DATA/GSE107011/abis_pbmc_data.csv')
abis_pbmc_label.to_csv('../DATA/GSE107011/abis_pbmc_label_15ct.csv')

In [56]:
abis_pbmc_label

,B Naive,B Ex,B NSM,B SM,Plasmablasts,T CD4,T CD8,Basophils LD,mDCs,pDCs,Monocytes C,Monocytes I,Monocytes NC,NK,Other
Sample Name,,,,,,,,,,,,,,,
453W,0.02750,0.00093,0.01560,0.0072,0.00066,0.25331,0.28258,0.06763,0.0038,0.00570,0.194,0.0141,0.0085,0.0678,0.05069
684C,0.06740,0.00270,0.01160,0.0106,0.00740,0.29714,0.15490,0.06464,0.0066,0.00610,0.196,0.0087,0.0172,0.0845,0.06452
CR3L,0.08380,0.00100,0.01020,0.0111,0.00540,0.26843,0.15129,0.04173,0.0076,0.00550,0.140,0.0083,0.0113,0.1080,0.14635
FLWA,0.05450,0.00180,0.01010,0.0158,0.00150,0.23634,0.08345,0.07061,0.0064,0.00420,0.196,0.0047,0.0079,0.1950,0.11170
FY2H,0.06020,0.00087,0.00390,0.0043,0.00110,0.19182,0.17287,0.03824,0.0099,0.00370,0.268,0.0455,0.0132,0.0260,0.16040
CYFZ,0.05690,0.00200,0.01080,0.0122,0.00180,0.17362,0.27818,0.10013,0.0038,0.00280,0.111,0.0289,0.0280,0.1400,0.04987
925L,0.12734,0.00127,0.01151,0.0156,0.00111,0.19425,0.18723,0.09335,0.0056,0.00099,0.170,0.0119,0.0147,0.0969,0.06825
9JD4,0.04700,0.00440,0.01120,0.0177,0.00270,0.30033,0.23596,0.04568,0.0048,0.00480,0.146,0.0113,0.0125,0.0753,0.08033
G4YW,0.03640,0.00280,0.01310,0.0181,0.00460,0.24426,0.13448,0.03759,0.0055,0.00270,0.186,0.0175,0.0167,0.1750,0.10527


In [55]:
abis_label2

,B Naive,B Ex,B NSM,B SM,Plasmablasts,T CD4,T CD8,Basophils LD,mDCs,pDCs,Monocytes C,Monocytes I,Monocytes NC,NK,Other
DZQV_CD8_naive,0.0000,0.00000,0.0000,0.0000,0.0000,0.00000,1.00000,0.00000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.00000
DZQV_CD8_CM,0.0000,0.00000,0.0000,0.0000,0.0000,0.00000,1.00000,0.00000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.00000
DZQV_CD8_EM,0.0000,0.00000,0.0000,0.0000,0.0000,0.00000,1.00000,0.00000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.00000
DZQV_CD8_TE,0.0000,0.00000,0.0000,0.0000,0.0000,0.00000,1.00000,0.00000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.00000
DZQV_MAIT,0.0000,0.00000,0.0000,0.0000,0.0000,0.00000,0.00000,0.00000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9JD4_PBMC,0.0470,0.00440,0.0112,0.0177,0.0027,0.30033,0.23596,0.04568,0.0048,0.0048,0.146,0.0113,0.0125,0.0753,0.08033
G4YW_PBMC,0.0364,0.00280,0.0131,0.0181,0.0046,0.24426,0.13448,0.03759,0.0055,0.0027,0.186,0.0175,0.0167,0.1750,0.10527
4DUY_PBMC,0.0519,0.00098,0.0120,0.0088,0.0061,0.24222,0.12392,0.10206,0.0056,0.0014,0.173,0.0059,0.0436,0.0792,0.14332
36TS_PBMC,0.0415,0.00210,0.0107,0.0131,0.0018,0.19150,0.17041,0.08545,0.0057,0.0051,0.247,0.0333,0.0350,0.0772,0.08014


In [42]:
np.sum(abis_label2, axis=1)

DZQV_CD8_naive    1.0
DZQV_CD8_CM       1.0
DZQV_CD8_EM       1.0
DZQV_CD8_TE       1.0
DZQV_MAIT         1.0
                 ... 
9JD4_PBMC         1.0
G4YW_PBMC         1.0
4DUY_PBMC         1.0
36TS_PBMC         1.0
CR3L_PBMC         1.0
Length: 126, dtype: float64

In [118]:
abis

,DZQV_CD8_naive,DZQV_CD8_CM,DZQV_CD8_EM,DZQV_CD8_TE,DZQV_MAIT,DZQV_VD2+,DZQV_VD2-,DZQV_TFH,DZQV_Treg,DZQV_Th1,...,G4YW_pDC,G4YW_mDC,G4YW_Neutrophils,G4YW_Basophils,925L_PBMC,9JD4_PBMC,G4YW_PBMC,4DUY_PBMC,36TS_PBMC,CR3L_PBMC
5_8S_rRNA,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5S_rRNA,15.517400,24.933300,0.000000,16.80640,4.769510,14.714600,13.880900,18.39910,17.975300,15.445400,...,13.794200,7.157120,8.076370,20.681800,14.184900,0.000000,11.085400,6.735800,6.145910,16.203500
7SK,2.589550,5.698690,2.584000,4.47815,6.029090,1.160110,7.539050,2.30605,3.941680,7.171800,...,7.972310,1.833560,4.134720,9.838080,4.694090,2.187640,2.756160,1.180040,4.908830,7.446450
A1BG,7.708026,4.807327,2.270340,2.99581,2.592271,1.031283,2.008449,2.84179,5.132423,3.168394,...,18.267554,14.227236,3.809277,3.990624,7.193748,7.536929,3.887696,2.235442,1.997778,3.808482
A1BG-AS1,11.379450,11.330981,4.699683,5.83558,8.761590,4.011840,4.393887,8.16249,4.707670,3.520375,...,12.937730,6.764657,0.097087,2.329913,7.407794,9.394410,6.499663,5.682043,5.969403,4.954699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11AP1,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ZYG11B,7.504700,7.434530,3.043490,3.00489,6.224360,3.746590,6.162452,6.65370,4.652842,6.030789,...,3.058625,4.855910,8.641030,2.742560,5.898185,7.249695,6.739620,5.399180,3.570432,3.656880
ZYX,12.409610,27.854160,65.023200,50.38362,60.364140,61.325510,53.687010,12.14344,67.534220,51.184640,...,128.725600,284.301400,989.887200,61.790070,249.604600,261.978300,290.415400,223.645200,207.546800,260.399600
ZZEF1,21.787460,31.670380,25.159950,17.44815,32.948070,10.891110,23.717420,28.22253,22.210690,30.843200,...,12.502900,12.949470,26.502740,20.313010,19.829610,12.152770,10.930280,19.782260,11.573840,10.919260


In [ ]:
abis[abis.columns[abis.columns.to_series().str.contains('_PBMC')]]